In [12]:
!uv pip install numpy pandas matplotlib tqdm ipywidgets datasets transformers evaluate

Resolved 64 packages in 455ms                                        
Prepared 1 package in 45ms                                               
Installed 1 package in 5ms0                                 
 + python-dotenv==1.1.0


In [2]:
!uv pip install --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cpu

Audited 3 packages in 4ms


In [3]:
!uv pip install google google-genai

Audited 2 packages in 6ms


In [13]:
!uv pip install python-dotenv

Audited 1 package in 12ms


In [14]:
from dotenv import load_dotenv
    
load_dotenv(dotenv_path='.env')

True

In [9]:
import json
import numpy as np
import pandas

import base64
import os
import textwrap

# Tools from Hugging Face
from datasets import load_dataset
from transformers import pipeline

from google import genai
from google.genai import types

In [5]:
def load_data(file_path):
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
            return data
    except FileNotFoundError:
        print(f"Error: File not found: {file_path}")
        return None
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in: {file_path}")
        return None

In [6]:
def generate_prompt(data):
    prompt = 'Answer the following question based on the given facts, rules, and preferences. Provide a true or false answer (binary cross-entropy).\n\n'
    
    # Add facts to the prompt
    prompt += 'Facts:\n'
    for fact in data['facts']:
        prompt += f'- {fact}\n'

    # Add rules to the prompt
    prompt += '\nRules:\n'
    for rule in data['rules']:
        prompt += f"- {rule}\n"
    
    # Add preferences to the prompt
    prompt += '\nPreferences:\n'
    for preference in data['preferences']:
        prompt += f'- {preference}\n'
    
    # Add the question
    prompt += f'\nQuestion: {data['question']}\n'

    return prompt

In [7]:
data = load_data('data/politics.json')[0]
prompt = generate_prompt(data)
print(prompt)

Answer the following question based on the given facts, rules, and preferences. Provide a true or false answer (binary cross-entropy).

Facts:
- F1: The Coastal Protection Act requires a 30% reduction in industrial emissions by 2030.
- F2: Senator Blackwood represents a state with significant coastal tourism.
- F3: Senator Blackwood's campaign received substantial funding from traditional energy companies.
- F4: Senator Blackwood has previously supported incremental environmental regulation.

Rules:
- R1: Senators typically prioritize economic interests of their major campaign donors.
- R2: Representatives of coastal regions generally support climate protection measures affecting their constituency.
- R3: Politicians who support incremental approaches typically oppose sweeping regulatory changes.

Preferences:
- P1: Regional economic interests take precedence over donor interests when directly conflicting.
- P2: Historical voting patterns predict future votes unless circumstances signi

In [17]:
def generate_response(prompt):
    client = genai.Client(
        api_key=os.environ.get('GEMINI_API_KEY'),
    )

    model = 'gemini-2.5-pro-exp-03-25'
    contents = [
        types.Content(
            role='user',
            parts=[
                types.Part.from_text(text=prompt),
            ],
        ),
    ]
    generate_content_config = types.GenerateContentConfig(
        response_mime_type='text/plain',
    )

    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        print(chunk.text, end='')

In [18]:
generate_response(prompt)

**Analysis:**

1.  **Pro-Vote Arguments (Supporting "True"):**
    *   F2 (Represents coastal state) + R2 (Coastal reps support climate protection) suggests a 'Yes'.
    *   P1 prioritizes regional interests (F2 - coastal tourism) over donor interests (F3 - energy companies), strengthening the R2 argument over the R1 argument.

2.  **Anti-Vote Arguments (Supporting "False"):**
    *   F3 (Energy funding) + R1 (Senators prioritize donors) suggests a 'No'. (However, P1 weakens this by prioritizing F2 over F3).
    *   F4 (Supports incremental regulation) + R3 (Incremental supporters oppose sweeping changes) suggests a 'No'. The 30% reduction in F1 sounds like a potentially "sweeping" change, not incremental.
    *   P2 (Historical patterns predict future votes) reinforces the F4+R3 argument. Blackwood's pattern is incrementalism, predicting opposition to non-incremental measures like the CPA.

3.  **Conflict Resolution:** The primary conflict is between the regional interest (F2+R2, stre